Imports

In [4]:
import ir_datasets
import logging
import sys
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jopec\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Configurando LOGGER

In [5]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout,
                    format='[%(asctime)s]%(levelname)s(%(name)s): %(message)s')
LOGGER = logging.getLogger(__name__)

Importando dataset

In [6]:
dataset = ir_datasets.load("highwire/trec-genomics-2006")

Processando map_consultas

In [8]:
map_consultas = {}

In [9]:
for query in dataset.queries_iter():
    # Usamos um f-string para juntar os dois campos com um espaço entre eles.
    text = query.text
    
    # Converte para minúsculas e tokeniza o texto
    processed_text = word_tokenize(text.lower())
    
    # Remove stopwords e palavras não alfabéticas, e junta as palavras novamente
    processed_text = ' '.join([word for word in processed_text if word.isalpha() and word not in stop_words])
    
    # Loga a consulta processada
    LOGGER.info(f"Processed query {query.query_id}: {processed_text}")
    
    # Armazena o resultado no dicionário
    map_consultas[query.query_id] = processed_text

[2025-07-22 03:10:58,506]INFO(__main__): Processed query 160: role prnp mad cow disease
[2025-07-22 03:10:58,507]INFO(__main__): Processed query 161: role ide alzheimer disease
[2025-07-22 03:10:58,507]INFO(__main__): Processed query 162: role cancer
[2025-07-22 03:10:58,508]INFO(__main__): Processed query 163: role apc adenomatous polyposis coli colon cancer
[2025-07-22 03:10:58,508]INFO(__main__): Processed query 164: role parkinson disease
[2025-07-22 03:10:58,509]INFO(__main__): Processed query 165: cathepsin ctsd apolipoprotein e apoe interactions contribute alzheimer disease
[2025-07-22 03:10:58,509]INFO(__main__): Processed query 166: role transforming growth cerebral amyloid angiopathy caa
[2025-07-22 03:10:58,510]INFO(__main__): Processed query 167: nucleoside diphosphate kinase contribute tumor progression
[2025-07-22 03:10:58,510]INFO(__main__): Processed query 168: regulate activity
[2025-07-22 03:10:58,511]INFO(__main__): Processed query 169: apc adenomatous polyposis coli

Salvando map_consultas como .json

In [81]:
with open('map_consultas.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_consultas,      # Dicionário de consultas processadas
        out_file_opened,    # Objeto do arquivo
        indent=2,           # Indentação para melhorar legibilidade
        ensure_ascii=False  # Permite caracteres não-ASCII
    )

Processando map_documentos

In [73]:
map_documentos = {}
map_index = {}

In [76]:
i = 1
for doc in dataset.docs_iter():
    fields_to_join = [
        doc.title,
        doc.journal
    ]
    
    # Junta todos os campos selecionados em uma única string, ignorando campos vazios (None)
    text = ' '.join(str(field) for field in fields_to_join if field)
    
    # Converte para minúsculas e tokeniza o texto
    processed_text_tokens = word_tokenize(text.lower())
    
    # Remove stopwords e palavras que não são puramente alfabéticas
    cleaned_tokens = [word for word in processed_text_tokens if word.isalpha() and word not in stop_words]
    
    # Junta os tokens limpos de volta em uma única string
    processed_text = ' '.join(cleaned_tokens)
    
    # Adiciona o documento processado ao dicionário, usando o doc_id como chave
    map_documentos[doc.doc_id] = processed_text
    map_index[doc.doc_id] = i
    i += 1
    

Salvando map_documentos como .json

In [82]:
with open('map_documentos.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_documentos,     # Dicionário de documentos processados
        out_file_opened,    # Objeto do arquivo
        indent=2,           # Indentação para melhorar legibilidade
        ensure_ascii=False  # Permite caracteres não-ASCII
    )

Processando map_consultas_relevancias

In [77]:
map_consultas_relevancias = {}

In [79]:
for qrel in dataset.qrels_iter():
    # Procura o ID numérico do documento (qrel.doc_id) no seu mapa de IDs.
    # Se o documento não foi indexado antes (ou seja, não está em map_index), .get() retorna None.
    doc_index = map_index.get(qrel.doc_id, None)

    # Executa o bloco somente se o documento do 'qrel' existir na nossa coleção indexada.
    if doc_index is not None:
        
        # Garante que a consulta (qrel.query_id) tenha uma entrada no dicionário.
        # Se a consulta é nova, cria um dicionário vazio {} para ela. Se já existe, não faz nada.
        # Esta é uma forma idiomática de inicializar um sub-dicionário.
        map_consultas_relevancias[qrel.query_id] = map_consultas_relevancias.get(qrel.query_id, {})

        # Esta linha é REDUNDANTE. O valor de 'doc_index' já foi obtido com segurança na primeira linha do loop.
        doc_index = map_index[qrel.doc_id]
        
        # Armazena a pontuação de relevância.
        # A estrutura final será: map_consultas_relevancias['id_da_consulta'][id_numerico_do_doc] = pontuação
        map_consultas_relevancias[qrel.query_id][doc_index] = qrel.relevance

Salvando map_consultas_relevancias como .json

In [84]:
with open('map_consultas_relevancias.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_consultas_relevancias, # Dicionário de relevâncias processadas
        out_file_opened,           # Objeto do arquivo
        indent=2,                  # Indentação para melhorar legibilidade
        ensure_ascii=False         # Permite caracteres não-ASCII
    )